In [2]:
import os
import getpass


os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

Add tools later

In [1]:
from tools.design_retriever import DesignRetrieverTool
from chains.design_rag import DesignRAG

# Initialize DesignRAG and create the tool
design_rag = DesignRAG()
design_retriever = DesignRetrieverTool(rag=design_rag)


tool_belt = [design_retriever]

Loaded 82 design documents


Pick a model good for chat and tools

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

model.bind_tools(tool_belt)

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10d8b4750>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10d8c1110>, root_client=<openai.OpenAI object at 0x10b08ef10>, root_async_client=<openai.AsyncOpenAI object at 0x10d8b4910>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': []}, config={}, config_factories=[])

Initialize state


In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Set up the nodes and graph


In [8]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, END

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)
uncompiled_graph.set_entry_point("agent")


def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
  "agent",
  should_continue
)
uncompiled_graph.add_edge("action", "agent")

graph = uncompiled_graph.compile()



Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'agent': Node(id='agent', name='agent', data=agent(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'action': Node(id='action', name='action', data=tools(tags=None, recurse=True, explode_args=False, func_accepts_config=True, func_accepts={'store': ('__pregel_store', None)}, tools_by_name={}, tool_to_state_args={}, tool_to_store_arg={}, handle_tool_errors=True, messages_key='messages'), metadata=None), '__end__': Node(id='__end__', name='__end__', data=<class 'langchain_core.utils.pydantic.LangGraphOutput'>, metadata=None)}, edges=[Edge(source='__start__', target='agent', data=None, conditional=False), Edge(source='action', target='agent', data=None, conditional=False), Edge(source='agent', target='action', data=None, conditional=True), Edge(source='agent', target='__end__', data=None, condi

Try it out!

In [11]:
from langchain_core.messages import HumanMessage

graph.invoke({"messages" : [HumanMessage(content="Hello, how are you?")]})

{'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='1af14df1-568c-460c-997f-11d59e70a3b3'),
  AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_709714d124', 'finish_reason': 'stop', 'logprobs': None}, id='run-d532debb-85a6-4730-9786-43543f1cfd43-0', usage_metadata={'input_tokens': 13, 'output_tokens': 31, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

Let's see if the RAG tool works.

In [ ]:


# Test the tool
test_requirements = {
    "style_description": "Modern and minimalist",
    "key_elements": ["clean lines", "white space", "typography"],
    "color_scheme": "Monochromatic with subtle accent colors",
    "layout_preferences": "Grid-based layout with clear hierarchy",
    "mood": "Professional and sophisticated"
}

# Create a test message
from langchain_core.messages import HumanMessage

test_message = HumanMessage(
    content="""I need a design that's modern and minimalist, with clean lines and plenty of white space. 
    I want it to use a monochromatic color scheme with subtle accent colors. 
    The layout should be grid-based with clear hierarchy. 
    The overall mood should be professional and sophisticated."""
)

# Invoke the graph with the test message
response = graph.invoke({"messages": [test_message]})
print(response)